In [80]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\920791\AppData\Local\Temp\ipykernel_66504\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [81]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [82]:
## Read Data for the Fraudulent Email Kaggle Challenge
#data = pd.read_csv("data\kg_train.csv",encoding='latin-1')

data = pd.read_csv("D:/OneDrive - Royal HaskoningDHV/920791/Pri 3/ironhack/lab-natural-language-processing/data/kg_train.csv", encoding='latin-1')


In [83]:
# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [84]:
# Your code
from sklearn.model_selection import train_test_split

In [85]:
# Print the column names to identify the correct ones
print(data.columns)

Index(['text', 'label'], dtype='object')


In [86]:
# Define features (X) and labels (y)
X = data['text']
y = data['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# Print the shapes of the resulting datasets
print('Train set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)


Train set: (800,) (800,)
Test set: (200,) (200,)


## Data Preprocessing

In [87]:
# Natural Language Toolkit library is not installed in my environment, then:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [88]:
#NLTK stopwords resource isn’t downloaded yet, so:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\920791\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [89]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [90]:
# Inline JavaScript and CSS are usually within <script> and <style> tags. We’ll use regex to find and remove these tags and their content.
import re

# Sample HTML content
html_content = """
<html>
<head>
<style>
body {background-color: powderblue;}
h1 {color: blue;}
p {color: red;}
</style>
</head>
<body>
<h1>This is a heading</h1>
<p>This is a paragraph.</p>
<script>
document.getElementById("demo").innerHTML = "Hello JavaScript!";
</script>
</body>
</html>
"""

# Remove inline JavaScript/CSS
cleaned_html = re.sub(r'<(script|style).*?>.*?</\1>', '', html_content, flags=re.DOTALL)
print(cleaned_html)


<html>
<head>

</head>
<body>
<h1>This is a heading</h1>
<p>This is a paragraph.</p>

</body>
</html>



In [91]:
# HTML comments are enclosed within <!-- and -->. We’ll use regex to find and remove these comments.
cleaned_html = re.sub(r'<!--.*?-->', '', cleaned_html, flags=re.DOTALL)
print(cleaned_html)


<html>
<head>

</head>
<body>
<h1>This is a heading</h1>
<p>This is a paragraph.</p>

</body>
</html>



In [92]:
# We’ll remove all remaining HTML tags to get plain text.
cleaned_html = re.sub(r'<.*?>', '', cleaned_html)
print(cleaned_html)







This is a heading
This is a paragraph.






- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [93]:
# Sample text
text = "Hello! This is a sample text with special characters: @#%&*() and numbers 12345. bThis is a test."

In [94]:
# 1- Remove all special characters
cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)
print(cleaned_text)

Hello This is a sample text with special characters  and numbers 12345 bThis is a test


In [95]:
# 2- Remove numbers
cleaned_text = re.sub(r'\d+', '', cleaned_text)
print(cleaned_text)

Hello This is a sample text with special characters  and numbers  bThis is a test


In [96]:
# 3- Remove all single characters
cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
print(cleaned_text)

Hello This is  sample text with special characters  and numbers  bThis is  test


In [97]:
# 4- Remove single characters from the start
cleaned_text = re.sub(r'^\w\s', '', cleaned_text)
print(cleaned_text)

Hello This is  sample text with special characters  and numbers  bThis is  test


In [98]:
# 5- Substitute multiple spaces with a single space
cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
print(cleaned_text)

Hello This is sample text with special characters and numbers bThis is test


In [99]:
# 6- Remove prefixed 'b'
cleaned_text = cleaned_text.replace('b', '')
print(cleaned_text)

Hello This is sample text with special characters and numers This is test


In [100]:
# 7- Convert to lowercase
cleaned_text = cleaned_text.lower()
print(cleaned_text)

hello this is sample text with special characters and numers this is test


## Now let's work on removing stopwords
Remove the stopwords.

In [101]:
# Sample text
text = "This is a sample text with some stopwords that need to be removed."

# Step 1: Define the list of stopwords (already done)
stop_words = set(stopwords.words('english'))

# Step 2: Split the text into words
words = text.split()

# Step 3: Remove stopwords
filtered_words = [word for word in words if word.lower() not in stop_words]

# Step 4: Join the words back into a single string
cleaned_text = ' '.join(filtered_words)

print(cleaned_text)

sample text stopwords need removed.


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [102]:
# 1- Import Necessary Libraries. Use the WordNetLemmatizer from the NLTK library to reduce words to their base form.

# Imports the WordNetLemmatizer class.
from nltk.stem import WordNetLemmatizer

# Imports the word_tokenize function to split text into words.
from nltk.tokenize import word_tokenize

# Downloads the WordNet data needed for lemmatization.
nltk.download('wordnet')

# Downloads the Punkt tokenizer models.
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\920791\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\920791\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [103]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\920791\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [104]:
# 2- Initialize the Lemmatizer. Create an instance of the lemmatizer.
lemmatizer = WordNetLemmatizer()

In [105]:
# 3- Tokenize the Text. Split the text into individual words.

# Sample text
text = "This is a sample text with running and jumps."

# Tokenize the text. word_tokenize(text): Splits the text into a list of words
words = word_tokenize(text)
print(words)

['This', 'is', 'a', 'sample', 'text', 'with', 'running', 'and', 'jumps', '.']


In [106]:
# 4- Lemmatize Each Word. Apply the lemmatizer to each word in the list.
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
print(lemmatized_words)

['This', 'is', 'a', 'sample', 'text', 'with', 'running', 'and', 'jump', '.']


In [107]:
# 5- Join the Words Back into a Single String with spaces in between.
cleaned_text = ' '.join(lemmatized_words)
print(cleaned_text)

This is a sample text with running and jump .


- This code will lemmatize the words in the text, reducing them to their base forms and creating cleaner data for analysis. 

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [108]:
# 1- Import Necessary Libraries
import pandas as pd

# CountVectorizer: Converts text data into a matrix of token counts.
from sklearn.feature_extraction.text import CountVectorizer

In [109]:
# 2- Load and Prepare the Data

# We create a sample DataFrame with text messages and their labels (ham or spam).
data = pd.DataFrame({
    'text': [
        "This is a ham message",
        "This is another ham message",
        "Spam message here",
        "Another spam message"
    ],
    'label': ['ham', 'ham', 'spam', 'spam']
})

#We then separate the messages into ham_messages and spam_messages
ham_messages = data[data['label'] == 'ham']['text']
spam_messages = data[data['label'] == 'spam']['text']

In [110]:
# 3- Initialize the CountVectorizer
# CountVectorizer is initialized with stop_words='english' to ignore common English stopwords.
vectorizer = CountVectorizer(stop_words='english')

In [111]:
# 4- Fit and Transform the Data
# Fit the Vectorizer to ham_messages and Transform Them into a Bag of Words Representation
ham_bow = vectorizer.fit_transform(ham_messages)

# Extract the Feature Names (Words) and Their Counts
ham_words = vectorizer.get_feature_names_out()
ham_word_counts = ham_bow.sum(axis=0).A1
ham_word_freq = dict(zip(ham_words, ham_word_counts))

# Repeat the Process for spam_messages
vectorizer = CountVectorizer(stop_words='english')
spam_bow = vectorizer.fit_transform(spam_messages)
spam_words = vectorizer.get_feature_names_out()
spam_word_counts = spam_bow.sum(axis=0).A1
spam_word_freq = dict(zip(spam_words, spam_word_counts))


In [112]:
# 5- Get the Top Words
top_ham_words = sorted(ham_word_freq.items(), key=lambda x: x[1], reverse=True)[:10]
top_spam_words = sorted(spam_word_freq.items(), key=lambda x: x[1], reverse=True)[:10]

print("Top 10 words in ham messages:", top_ham_words)
print("Top 10 words in spam messages:", top_spam_words)

Top 10 words in ham messages: [('ham', np.int64(2)), ('message', np.int64(2))]
Top 10 words in spam messages: [('message', np.int64(2)), ('spam', np.int64(2))]


## Extra features

In [117]:
import re

# Define the clean_text function
def clean_text(text):
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove all special characters
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\b\w\b', '', text)  # Remove all single characters
    text = re.sub(r'^\w\s', '', text)  # Remove single characters from the start
    text = re.sub(r'\s+', ' ', text).strip()  # Substitute multiple spaces with a single space
    text = text.replace('b', '')  # Remove prefixed 'b'
    text = text.lower()  # Convert to lowercase
    return text

In [118]:
from sklearn.model_selection import train_test_split

# Assuming you have your data loaded in a DataFrame called 'data'
# Define features (X) and labels (y)
X = data['text']
y = data['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# Create DataFrames for training and validation sets
data_train = pd.DataFrame({'text': X_train, 'label': y_train})
data_val = pd.DataFrame({'text': X_test, 'label': y_test})

In [119]:
# Apply the preprocessing function to both training and validation sets
data_train['preprocessed_text'] = data_train['text'].apply(clean_text)
data_val['preprocessed_text'] = data_val['text'].apply(clean_text)

In [120]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
1,This is another ham message,ham,this is another ham message,1,0,27
3,Another spam message,spam,another spam message,1,0,20
2,Spam message here,spam,spam message here,1,0,17


## How would work the Bag of Words with Count Vectorizer concept?

In [122]:
from scipy.sparse import hstack

In [124]:
import numpy as np

In [125]:
# Apply CountVectorizer to the preprocessed text
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_counts = vectorizer.transform(data_val['preprocessed_text'])

# Combine with additional indicators
X_train_combined = hstack([X_train_counts, np.array(data_train[['money_mark', 'suspicious_words', 'text_len']])])
X_val_combined = hstack([X_val_counts, np.array(data_val[['money_mark', 'suspicious_words', 'text_len']])])

In [126]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Train the model
model = MultinomialNB()
model.fit(X_train_combined, data_train['label'])

# Predict and evaluate
y_pred = model.predict(X_val_combined)
print("Accuracy:", accuracy_score(data_val['label'], y_pred))
print(classification_report(data_val['label'], y_pred))

Accuracy: 1.0
              precision    recall  f1-score   support

         ham       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [128]:
# 1- Load the Vectorizer: First, we need to import the TfidfVectorizer from sklearn.feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer

In [129]:
# 2- Vectorize the Entire Dataset: Next, we’ll create an instance of TfidfVectorizer and use it to transform the text data into TF-IDF features.
# The data loaded in a DataFrame called 'data'
# Define features (X)
X = data['text']

# Step 1: Load the vectorizer
vectorizer = TfidfVectorizer()

# Step 2: Vectorize the entire dataset
X_tfidf = vectorizer.fit_transform(X)


In [130]:
# 3- Print the shape of the vectorized dataset
print("Shape of the vectorized dataset:", X_tfidf.shape)


Shape of the vectorized dataset: (4, 7)


## And the Train a Classifier?

In [131]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the validation data
X_val_tfidf = vectorizer.transform(X_test)

In [132]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the classifier
classifier = MultinomialNB()

# Train the classifier
classifier.fit(X_train_tfidf, y_train)


MultinomialNB()

In [133]:
from sklearn.metrics import accuracy_score, classification_report

# Predict the labels for the validation set
y_pred = classifier.predict(X_val_tfidf)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print the classification report
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

         ham       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [1]:
# 1- Import Libraries:

# Data Manipulation:
# pandas: Provides data structures like DataFrames, which are useful for handling and processing structured data.
    
import pandas as pd
    
# Feature Extraction:
# CountVectorizer: Converts a collection of text documents to a matrix of token counts.
# TfidfVectorizer: Converts a collection of raw documents to a matrix of TF-IDF features, which reflect the importance of words in the documents.

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

# Model Training:
# MultinomialNB: Implements the Multinomial Naive Bayes algorithm, which is suitable for classification with discrete features (like word counts for text classification).

from sklearn.naive_bayes import MultinomialNB

# Model Evaluation:
# accuracy_score: Calculates the accuracy of the model by comparing the predicted labels with the true labels.

from sklearn.metrics import accuracy_score

In [30]:
# 2a- Load Dataset: We load the dataset from the provided URL. The dataset contains messages and their labels (spam or ham). Fraudulent Email Kaggle Challenge
# The latin-1 encoding, also known as ISO-8859-1, is a character encoding standard used to represent text in Western European languages. It includes characters for English, French, German, Spanish, and other Western European languages.

data = pd.read_csv("D:/OneDrive - Royal HaskoningDHV/920791/Pri 3/ironhack/lab-natural-language-processing/data/kg_train.csv", encoding='latin-1')

In [31]:
# 2b- Verify Data Loading: Print the first few rows of the dataset to ensure it is loaded correctly.

print(data.head())

                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0


In [32]:
# 3a- Preprocess Data: We convert the labels from text (‘ham’ and ‘spam’) to numerical values (0 and 1) to make them suitable for model training. 0 for ham and 1 for spam

# Check for missing values in the dataset

print(data.isnull().sum())

# Drop rows with missing values in the 'text' or 'label' columns

data = data.dropna(subset=['text', 'label'])

# Convert the labels to numerical values (0 for ham and 1 for spam)

data['label'] = data['label'].map({'ham': 0, 'spam': 1})


text     0
label    0
dtype: int64


There were 5964 missing values in the label column.

In [33]:
# 3b- Check Data After Preprocessing

print(data.head())
print(data.shape)

                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...    NaN
1                                           Will do.    NaN
2  Nora--Cheryl has emailed dozens of memos about...    NaN
3  Dear Sir=2FMadam=2C I know that this proposal ...    NaN
4                                                fyi    NaN
(5964, 2)


In [24]:
# 4- Split Data: We split the dataset into training and testing sets. The training set is used to train the model, and the testing set is used to evaluate the model’s performance.

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [7]:
# 5- Feature Extraction: We use two methods to convert the text data into numerical features:

# 5a- Method 1: Count Vectorizer: Converts text into a matrix of token counts.

count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# 5b- Method 2: TF-IDF Vectorizer: Converts text into a matrix of TF-IDF features, which reflect the importance of words in the documents.

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# 6- Train and Evaluate Model (Count Vectorizer): We train the MultinomialNB model using the Count Vectorizer features and evaluate its accuracy on the test set.

model_count = MultinomialNB()
model_count.fit(X_train_count, y_train)
y_pred_count = model_count.predict(X_test_count)
accuracy_count = accuracy_score(y_test, y_pred_count)